# Lab 8 Supervised Fine Tuning

In this lab, we will perform parameter efficient finetuning (PEFT) to finetune a llama-2 model, using the HuggingFace SFTTrainer tool from its trl library.


## 0. Dependencies and compatibility

In [1]:
!pip install -r requirements.txt

In [2]:
# Test whether your GPU supports bfloat16
import torch
major, _ = torch.cuda.get_device_capability()
if major >= 8:
    print("""Your GPU supports bfloat16: you can accelerate training by setting 
          bnb_4bit_compute_dtype to torch.bfloat16 and bf16 in the trainer to True""")

## 1. Prepare the dataset

See the notebook 01_prepare_data.ipynb

Make sure that you generate the correct dataset using that notebook before continuing. 

## 2. Get the original inference results (baseline)

Here we perform an inference on the original llama-2 model.

### 2.1 Setting up the inference (same as previous labs)

In [3]:
################################################################################
# Shared parameters between inference and SFT training
################################################################################

# The base model
model_name = "/share/model/llama-2-7b-chat-hf"
# Use a single GPU
# device_map = {'':0}
# Use all GPUs
device_map = "auto"


In [4]:
################################################################################
# bitsandbytes parameters
################################################################################
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,    # use 4-bit precision for base model loading
    bnb_4bit_quant_type= "nf4",  # Quantization type (fp4 or nf4)
    bnb_4bit_compute_dtype= torch.bfloat16,   # Compute dtype for 4-bit base models  "float16" or torch.bfloat16
    bnb_4bit_use_double_quant= False,  # Activate nested quantization for 4-bit base models (double quantization)
)

In [5]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)

# Load base model with bnb config
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


In [6]:
# Run text generation pipeline with our next model
prompt = "Write a Python function to return the mode (the value or values that appear most frequently within that list) in a given list. If there are multiple modes, return -1. You should generate a function:\n def solve(list):"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

### 2.2 Evaluating the code generated

In [7]:
# Using regrex to capture the generated Python to a string
import re

def extract_first_code_snippet(text):
    # Use a regular expression to find the first code snippet enclosed in triple backticks
    match = re.search(r"```(.*?)```", text, re.S)
    if match:
        # Return the first matched group, which is the content within the backticks
        return match.group(1)
    else:
        # Return None if no match is found
        return None
    
code = extract_first_code_snippet(result[0]['generated_text'])
print(code)

In [8]:
# Define a testcase using the standard python unittest library

import unittest

# place holder for the AI generated code
def solve(list):
    return 0  

class TestGeneratedCode(unittest.TestCase):

    def test_no_single_mode(self):
        self.assertEqual(solve([3, 2, 1]), -1)

    def test_single_mode(self):
        self.assertEqual(solve([4, 9, 2, 33, 2]), 2)

    def test_no_single_mode_3(self):
        self.assertEqual(solve([7, 9, 11, 323, 996]), -1)

def run_all_tests():
    unittest.main(argv=[''], verbosity=2, exit=False) 

In [9]:
exec(code)  # run the generated code to redefine solve() function
run_all_tests() # Expect to fail

## 3. Finetune the Llama2 model
In this section, we will finetune a chat model. We choose Llama-chat-7B as our base model and our dataset is code instrcutions, which guides LLM to write programming code from given natural language. Don't panic since this is not a difficult task and we will separate this task into several procedures.

### 3.1  Set training arguments
In this subsection, you need to read the given code snippets below. If you have some questions, you can either refer to the official documents or discuss with TAs or you classmates.

In [10]:
################################################################################
# Model name and directories
################################################################################

# The base model
model_name = "/share/model/llama-2-7b-chat-hf"
# The instruction dataset to use
dataset_name = "/scratch2/py18k"
# Fine-tuned model name
new_model = "/scratch2/llama-2-7b-py18k"
# Output directory where the model predictions and checkpoints will be stored
output_dir = "/scratch2/results"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64
# Alpha parameter for LoRA scaling
lora_alpha = 16
# Dropout probability for LoRA layers
lora_dropout = 0.1
bias="none"
task_type="CAUSAL_LM"

################################################################################
# Training parameters (passed to TrainingArguments)
################################################################################

# Number of training epochs
num_train_epochs = 1
# Number of training steps (overrides num_train_epochs)
# max_steps = 100
# Enable fp16/bf16 training (set bf16 to True if supported by your GPU)
fp16 = False
bf16 = True
# Batch size per GPU for training
per_device_train_batch_size = 4
# Batch size per GPU for evaluation
per_device_eval_batch_size = 4
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1
# Enable gradient checkpointing
gradient_checkpointing = True
# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4
# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001
# Optimizer to use
optim = "paged_adamw_32bit"
# Learning rate schedule
lr_scheduler_type = "cosine"
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
# Save checkpoint every X updates steps
save_steps = 0

################################################################################
# Monitoring parameters
################################################################################

# Logging dir (for tensorboard)
logging_dir = f"{output_dir}/logs"
# Log every X updates steps
logging_steps = 25
# Monitoring and Visualizing tools
report_to = "tensorboard"

################################################################################
# SFTTrainer parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 512
# Pack multiple short examples in the same input sequence to increase efficiency
packing = False


### 3.2 Load Training Dataset (prepared in section 1)

In [11]:
# Load dataset
from datasets import load_from_disk
dataset = load_from_disk(dataset_name)
print(dataset[0])

### 3.3 Finetuning the model

### 3.3.1 Construct the configuration objects

In [12]:
from peft import PeftModel
from trl import SFTTrainer

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    #max_steps=max_steps,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    logging_steps=logging_steps,
    logging_dir=logging_dir,
    report_to=report_to, 
)

In [13]:
from peft import LoraConfig
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias=bias,
    task_type=task_type,
)

### 3.3.2 Initialize SFTTrainer

In [14]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

### 3.3.3 start the tensorboard for monitoring

In [15]:
import tensorboard

# Visualize the finetuning process.
#%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir /scratch2/results/logs

### 3.3.4 Train the model (takes about half an hour on 4090)

In [16]:
# # Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

In [17]:
# release GPU memory here

torch.cuda.reset_peak_memory_stats()
torch.cuda.empty_cache()

### 3.3.5 Merge Lora_model with Base model and save the merged model

You should first export Lora model with base model and convert them into hf checkpoint. 
This makes up the final trained merged model.

In [18]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
merged_model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Save merged model to disk (optional)
merged_model.save_pretrained(f'{new_model}_merged')


In [19]:
# (Optional) Push Lora model to Huggingface hub (optional)
# !huggingface-cli login

# merged_model.push_to_hub(new_model, use_temp_dir=False)
# tokenizer.push_to_hub(new_model, use_temp_dir=False)

### 3.6 examine the results in tensorboard


In [20]:
# set up port forwarding in vscode
# open the tensorboard page at http://localhost:6006

### 3.7 Test the SFTed model

In [21]:
# Run text generation pipeline with our next model
prompt = "Write a Python function to return the mode (the value or values that appear most frequently within that list) in a given list. If there are multiple modes, return -1. You should generate a function:\n def solve(list):"
pipe = pipeline(task="text-generation", model=merged_model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [22]:
code = extract_first_code_snippet(result[0]['generated_text'])
print(code)


In [23]:
exec(code)
run_all_tests() # Expect to pass